In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=" "

In [2]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
sess = tf.Session()

In [3]:
from loss import *

In [3]:
a = np.random.randint(0, 8, [6, 16, 16, 6])*1.0#tf.random.uniform([6, 320, 256, 12], maxval=vmax+1, dtype='int32')
b = np.random.randint(0, 10, [6, 16, 16, 6])*1.0#tf.random.uniform([6, 320, 256, 12], maxval=vmax+1, dtype='int32')

In [5]:
def Custom_SSIM(y_true, y_pred):
    """
    y_true : [batch, height, width, channel]
    y_pred : [batch, height, width, channel]
    """
    # b, h, w, c = tf.shape(y_true)
    #print(tf.shape(y_true))

    # [b, h, w, c] -> [b*c, h, w]
    tmp_true = tf.reshape(tf.transpose(y_true, [0, 3, 1, 2]),
                          [tf.shape(y_true)[0]*tf.shape(y_true)[3], tf.shape(y_true)[1], tf.shape(y_true)[2], 1])
    tmp_pred = tf.reshape(tf.transpose(y_pred, [0, 3, 1, 2]),
                          [tf.shape(y_true)[0]*tf.shape(y_true)[3], tf.shape(y_true)[1], tf.shape(y_true)[2], 1])
    max_val = tf.reduce_max(tmp_true, axis=(1, 2, 3), keepdims=True)-tf.reduce_min(tmp_true, axis=(1, 2, 3), keepdims=True)
    
    ssim = tf.image.ssim(tmp_true, tmp_pred, max_val=max_val)
    
    # ssim : [b*c, 1] -> [b, c]
    ssim = tf.reshape(ssim, [tf.shape(y_true)[0], tf.shape(y_true)[3]])
    
    return (1.-ssim)/2.

In [6]:
ph1 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float64)
ph2 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float64)

ssim = Custom_SSIM(ph1, ph2)

In [45]:
sess_test = sess.run(ssim, feed_dict={ph1:b, ph2:a})

In [46]:
sess_test

array([[0.53628373, 0.5066118 , 0.483469  , 0.49045593, 0.45590618,
        0.5755512 ],
       [0.47619876, 0.6129287 , 0.52747065, 0.53778297, 0.44359028,
        0.4475478 ],
       [0.49658242, 0.4689433 , 0.5410947 , 0.4720045 , 0.46674955,
        0.49124426],
       [0.35568428, 0.5780693 , 0.46426848, 0.5180951 , 0.45292622,
        0.53033787],
       [0.49299815, 0.57605255, 0.47505072, 0.48900303, 0.53030723,
        0.6142926 ],
       [0.5248959 , 0.5601461 , 0.5389449 , 0.47243908, 0.4850606 ,
        0.5661313 ]], dtype=float32)

In [43]:
f_a = np.reshape(np.transpose(a, [0, 3, 1, 2]), [36, 16, 16, 1])
f_b = np.reshape(np.transpose(b, [0, 3, 1, 2]), [36, 16, 16, 1])

In [41]:
ph3 = tf.placeholder(shape=[None, None, None], dtype=tf.float64)
ph4 = tf.placeholder(shape=[None, None, None], dtype=tf.float64)

tf_ssim = tf.image.ssim(ph3, ph4, max_val=1)


In [44]:
idx = -1
(1-sess.run(tf_ssim, feed_dict={ph3:f_a[idx]/f_b[idx].max(), ph4:f_b[idx]/f_b[idx].max()}))/2

0.5661313459277153

In [19]:
def mutual_information_single(hist2d):
    tmp = tf.cast(hist2d, dtype='float64')
    pxy = tmp / tf.reduce_sum(tmp)
    px = tf.reduce_sum(pxy, axis=1)
    py = tf.reduce_sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = tf.greater(pxy, 0)
    return tf.reduce_sum(tf.boolean_mask(pxy, nzs) * tf.log(tf.boolean_mask(pxy, nzs) / tf.boolean_mask(px_py, nzs)))

def tf_joint_histogram(y_true, y_pred):
    """
    y_true : [batch, height, width, channel]
    y_pred : [batch, height, width, channel]
    """
    #print("joint1")
    vmax = 255
    #b, h, w, c = tf.shape(y_true)
    
    
    # Intensity Scaling
    max_int = tf.reduce_max(y_true, axis = [1,2], keepdims=True)
    tmp_true = tf.round(y_true / max_int * vmax)
    tmp_pred = tf.round(y_pred / max_int * vmax)
    
    #print("joint2")
    # [batch, height, width, channel]
    # -> [batch, height * width, channel]
    # -> [batch, channel, height * width]
    flat_true = tf.transpose(tf.reshape(tmp_true,
                                        [tf.shape(y_true)[0], tf.shape(y_true)[1]*tf.shape(y_true)[2], tf.shape(y_true)[-1]]), [0, 2, 1])
    flat_true = tf.reshape(flat_true, [tf.shape(y_true)[0]*tf.shape(y_true)[-1], tf.shape(y_true)[1]*tf.shape(y_true)[2]])
    flat_pred = tf.transpose(tf.reshape(tmp_pred, [tf.shape(y_true)[0], tf.shape(y_true)[1]*tf.shape(y_true)[2], tf.shape(y_true)[-1]]), [0, 2, 1])
    flat_pred = tf.reshape(flat_pred, [tf.shape(y_true)[0]*tf.shape(y_true)[-1], tf.shape(y_true)[1]*tf.shape(y_true)[2]])
    #print("joint3")
    output = (flat_pred * (vmax+1)) + (flat_true+1)
    #print("joint4")
    # [b*c, 65536]
    output = tf.map_fn(lambda x : tf.cast(tf.histogram_fixed_width(x, value_range=[1, (vmax+1)**2], nbins=(vmax+1)**2), 'float32'), output)
    # [b, c, 256, 256] -> [b, 256, 256, c]
    output = tf.transpose(tf.reshape(output, [tf.shape(y_true)[0], tf.shape(y_true)[-1], vmax+1, vmax+1]), [0, 2, 3, 1])
    #print("joint5")
    return output, y_true, y_pred

def mutual_information(y_true, y_pred):
    """
    y_true : [batch, height, width, channel]
    y_pred : [batch, height, width, channel]
    """
    # [b, 256, 256, c]
    joint_histogram, _, _ = tf_joint_histogram(y_true, y_pred)
    #b, h, w, c = tf.shape(joint_histogram)
    #print("mutual1")
    # [b*c, 256, 256]
    reshape_joint_histogram = tf.reshape(tf.transpose(joint_histogram, [0, 3, 1, 2]), [tf.shape(joint_histogram)[0]*tf.shape(joint_histogram)[-1], tf.shape(joint_histogram)[1], tf.shape(joint_histogram)[2]])
    #print("mutual2")
    output = tf.map_fn(lambda x : mutual_information_single(x), reshape_joint_histogram, dtype=tf.float64)
    #print("mutual3")
    output = tf.reshape(output, [tf.shape(joint_histogram)[0], tf.shape(joint_histogram)[-1]])
    return tf.cast(-tf.reduce_mean(output, axis=1), 'float32')

In [20]:
ph1 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float32)
ph2 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float32)

In [23]:
hist2d = mutual_information(ph1, ph2)

In [24]:
test = sess.run(hist2d, feed_dict={ph1:a, ph2:b})

In [25]:
test

array([0.7322901 , 0.7136585 , 0.7664927 , 0.7276379 , 0.7035092 ,
       0.71937126], dtype=float32)